In [1]:
%pip install elasticsearch==7.17.4

In [1]:
import pandas as pd
from elasticsearch import Elasticsearch

In [3]:
#!pip install gdown --user
#!pip install --upgrade gdown

In [6]:
df = pd.read_json('data/semanticscholar.json')
df.head()

,paperId,url,title,abstract,year,referenceCount,citationCount,fieldsOfStudy,authors
0,9af8f2bba51c76fe7fcc1eec5b57b5445747b5ad,https://www.semanticscholar.org/paper/9af8f2bb...,GATE: Graph Attention Transformer Encoder for ...,Prevalent approaches in cross-lingual relation...,2020.0,81,21,['Computer Science'],"[{'authorId': '38123220', 'name': 'Wasi Uddin ..."
1,0eff37167876356da2163b2e396df2719adf7de9,https://www.semanticscholar.org/paper/0eff3716...,CrossViT: Cross-Attention Multi-Scale Vision T...,The recently developed vision transformer (ViT...,2021.0,55,152,['Computer Science'],"[{'authorId': '48239920', 'name': 'Chun-Fu Che..."
2,7c5c149699a0ba54b52cd5b9e291077f4a1f9d13,https://www.semanticscholar.org/paper/7c5c1496...,Synthesizer: Rethinking Self-Attention in Tran...,The dot product self-attention is known to be ...,2020.0,32,136,['Computer Science'],"[{'authorId': '144447820', 'name': 'Yi Tay'}, ..."
3,355601688c5c567bc5900c2ed09941538bfed960,https://www.semanticscholar.org/paper/35560168...,Molecule Attention Transformer,Designing a single neural network architecture...,2020.0,58,49,"['Computer Science', 'Physics', 'Mathematics']","[{'authorId': '51455558', 'name': 'Lukasz Mazi..."
4,5d1aadd4053ba4fdcd71fe1c87a53b7540ef4eea,https://www.semanticscholar.org/paper/5d1aadd4...,Multimodal Co-Attention Transformer for Surviv...,Survival outcome prediction is a challenging w...,2021.0,74,9,['Computer Science'],"[{'authorId': '2108279369', 'name': 'Richard J..."


In [10]:
ES_NODES = "http://localhost:9200"
df = pd.read_json('semanticscholar.json')


def prepare_es_data(index, df):
    records = df.to_dict(orient="records")
    es_data = []
    for idx, record in enumerate(records):
        meta_dict = {
            "index": {
                "_index": index,
                "_id": idx
            }
        }
        es_data.append(meta_dict)
        es_data.append(record)
    return es_data

def index_es_data(index, es_data):
    es_client = Elasticsearch(hosts = [ES_NODES])
    if es_client.indices.exists(index=index):
        print("deleting the '{}' index.".format(index))
        res = es_client.indices.delete(index=index)
        print("Response from server: {}".format(res))

    print("creating the '{}' index.".format(index))
    res = es_client.indices.create(index=index)
    print("Response from server: {}".format(res))

    print("bulk index the data")
    res = es_client.bulk(index=index, body=es_data, refresh = True)
    print("Errors: {}, Num of records indexed: {}".format(res["errors"], len(res["items"])))


es_data = prepare_es_data(index="articles", df=df)
index_es_data(index="articles", es_data=es_data)

deleting the 'articles' index.
Response from server: {'acknowledged': True}
creating the 'articles' index.
Response from server: {'acknowledged': True, 'shards_acknowledged': True, 'index': 'articles'}
bulk index the data
Errors: True, Num of records indexed: 7946


In [14]:
es_client = Elasticsearch(hosts = [ES_NODES])
query = 'dispensing with recurrence and convolutions entirely'
query_response = es_client.search(index="articles", query={"match": {"abstract": query}})

In [15]:
query_response['hits']['hits'][:3]

[{'_index': 'articles',
  '_type': '_doc',
  '_id': '5393',
  '_score': 23.776892,
  '_source': {'paperId': '40388be98979df08d4f9457a6e8f51103fa4e003',
   'url': 'https://www.semanticscholar.org/paper/40388be98979df08d4f9457a6e8f51103fa4e003',
   'title': 'A language model for Amdo Tibetan speech recognition',
   'abstract': 'We built a language model which is based on Transformer network architecture, used attention mechanisms to dispensing with recurrence and convalutions entirely. Through the transliteration of Tibetan to International Phonetic Alphabets, the language model was trained using the syllables and phonemes of the Tibetan word as modeling units to predict corresponding Tibetan sentences according to the context semantics of IPA. And it combined with the acoustic model as the Tibetan speech recognition was compared with end-to-end Tibetan speech recognition.',
   'year': 2021.0,
   'referenceCount': 24,
   'citationCount': 0,
   'fieldsOfStudy': "['Computer Science']",
   